In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. PyTorch is using GPU.")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. PyTorch is using CPU.")

CUDA is available. PyTorch is using GPU.
Device name: NVIDIA T400


In [ ]:
%reset -f

from transformers import AutoModelForSequenceClassification
from transformers import XLMRobertaTokenizer, AutoConfig , AutoTokenizer
import numpy as np
from scipy.special import softmax

 

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"



tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL )

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL )

tokenizer.save_pretrained(MODEL)

model.save_pretrained(MODEL)



config.json:   0%|          | 0.00/982 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [3]:
import time

start_time = time.time()

text = "i m  sad"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

end_time = time.time()
print(f"Execution time: {end_time - start_time:.4f} seconds")

1) negative 0.899
2) neutral 0.0823
3) positive 0.0187
Execution time: 0.1399 seconds


## good one 

In [ ]:
%reset -f

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load model and tokenizer
model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=".")
model = AutoModelForSequenceClassification.from_pretrained(model_name, cache_dir=".")

# Text input
text =  "كانت الرحلة عادية"

# Tokenize input
inputs = tokenizer(text, return_tensors="pt")

# Get model output
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to probabilities
probs = F.softmax(logits, dim=-1).squeeze()

# Define class labels
labels = [
    "Very Negative",
    "Negative",
    "Neutral",
    "Positive",
    "Very Positive"
]

# Print probabilities for each class
for label, prob in zip(labels, probs):
    print(label)
    print(round(prob.item(), 3))



tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Very Negative
0.036
Negative
0.066
Neutral
0.824
Positive
0.053
Very Positive
0.019
